In [ ]:
################################################################################
#                                                                              #
#    ██╗    ██╗██████╗  █████╗      ██████╗ ██████╗ ██╗      █████╗ ██████╗    #
#    ██║    ██║██╔══██╗██╔══██╗    ██╔════╝██╔═══██╗██║     ██╔══██╗██╔══██╗   #
#    ██║ █╗ ██║██████╔╝███████║    ██║     ██║   ██║██║     ███████║██████╔╝   #
#    ██║███╗██║██╔═══╝ ██╔══██║    ██║     ██║   ██║██║     ██╔══██║██╔══██╗   #
#    ╚███╔███╔╝██║     ██║  ██║    ╚██████╗╚██████╔╝███████╗██║  ██║██████╔╝   #
#     ╚══╝╚══╝ ╚═╝     ╚═╝  ╚═╝     ╚═════╝ ╚═════╝ ╚══════╝╚═╝  ╚═╝╚═════╝    #
#                                                                              #
#    Version: 1.0 (15/01/2022)                                                 #
#                                                                              #
################################################################################

# Configure
Please change in resources the execution environment type to: GPU

In [ ]:
################################################################################
#                                                                              #
#    LAST TEST:                                                                #
#    Hashcat v6.2.5-116-g5e69729f1                                             #
#    hcxpcapngtool 6.2.5-22-gdbe1f19                                           #
#                                                                              #
################################################################################

In [ ]:
# 1.- Check GPU Specs
!nvidia-smi

In [ ]:
# 2.- Install Hashcat and hcxtools

# Clone, build and install Hashcat
!apt install cmake build-essential -y
!apt install checkinstall git -y
!git clone https://github.com/hashcat/hashcat.git
!cd hashcat && git submodule update --init && make && make install

# Clone, build and install hcxtools
!mkdir /root/hcxtools
!git clone https://github.com/ZerBea/hcxtools.git
!cd hcxtools && make && make install

In [ ]:
# OPTIONAL: Check the Hashcat Version to make sure it's installed correctly
!hashcat --version
# OPTIONAL: Check the Hcxptools Version to make sure it's installed correctly
!hcxpcapngtool --version

## >> Banchmark (Optional)

In [ ]:
# OPTIONAL: Check WPA2 EAPOL Benchmark (change this to check the speed of whichever hash you're cracking)
!hashcat -w 4 -m 22000 --benchmark

# LOCAL VERSION

## >> Local Upload File
Dir CAP: /content/cap_files | Dir HC22000: /content/hc22000_files

In [ ]:
# 3.- Upload ONLY CAP FILES

from google.colab import files
!mkdir cap_files
!mkdir hc22000_files
uploaded = files.upload()

for f_name in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=f_name, length=len(uploaded[f_name])))

!mv {f_name} "/content/cap_files/{f_name}"

In [ ]:
# OPTIONAL: Check that your hashes are accessible to Colab

!ls /content/cap_files/

In [ ]:
# 4.- Convert CAP File to HC22000 file.

!hcxpcapngtool -o /content/hc22000_files/test.hc22000 /content/cap_files/cap.cap

In [ ]:
# OPTIONAL: Check that your hashes are accessible to Colab

!ls /content/hc22000_files/

## >> Local Brute Force Attack

In [ ]:
# 5.- Start pure brute force cracking

# Crack with a generate wordlist for charset
!hashcat --force -m 22000 -a 3 -w 1 /content/hc22000_files/test.hc22000 014?d?d?d?d?d?d?d --session TEST

In [ ]:
# OPTIONAL: Restore session TEST
!hashcat --session TEST --restore

# GDRIVE VERSION

## >> GDrive File 
Make a directory called 'wpacolab' in your Google Drive before this

In [ ]:
# 3.- Get CAP file for Google Drive

from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/My\ Drive/wpacolab
!ln -s /content/drive/My\ Drive/wpacolab /root/wpacolab

In [ ]:
# 4.- Convert CAP File to HC22000 file.

!hcxpcapngtool -o /content/drive/MyDrive/hashcolab/test.hc22000 /content/drive/MyDrive/hashcolab/cap.cap

## >> GDrive Brute Force Attack

In [ ]:
# 5.- Start pure brute force cracking with a session name

# Crack with a generate wordlist for charset
!hashcat --force -m 22000 -a 3 -w 1 /content/drive/MyDrive/hashcolab/test.hc22000 ?d?d?d?d?d?d?d --session TEST

In [ ]:
# OPTIONAL: Restore session TEST
!hashcat --session TEST --restore

# MULTIPLE WORDLIST USING MASK ATTACK VERSION

In [ ]:
# 5.- Create list of different types of charsets

import subprocess
import re
from tqdm import tqdm
is_cracked = False
charlist = list()

# Characters
# 
# The following built-in character sets exist:
# 
# ?l = abcdefghijklmnopqrstuvwxyz
# ?u = ABCDEFGHIJKLMNOPQRSTUVWXYZ
# ?d = 0123456789
# ?h = 0123456789abcdef
# ?H = 0123456789ABCDEF
# ?s = «space»!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
# ?a = ?l?u?d?s
# ?b = 0x00 - 0xff

# ?d?d?d?d?d?d?d?d = 00000000 > 99999999
charlist.append('?d?d?d?d?d?d?d?d')
# ?l?l?l?l?l?l?l?l = aaaaaaaa > zzzzzzzz
charlist.append('?l?l?l?l?l?l?l?l')
# ?h?h?h?h?h?h?h?h = 00000000 > ffffffff
charlist.append('?h?h?h?h?h?h?h?h')
# ?H?H?H?H?H?H?H?H = 00000000 > FFFFFFFF
charlist.append('?H?H?H?H?H?H?H?H')

print(*charlist)

In [ ]:
# 6.- Masive Attack

def check_if_cracked(command): 
  output = subprocess.check_output([command + " --show"], shell=True).decode()
  if len(output) == 0:
    return False
  else:
    return True

for wordlist in tqdm(charlist):
  command = 'hashcat --force -m 22000 -a 3 -w 1' + ' /content/hc22000_files/test.hc22000 ' + wordlist
  subprocess.call([command],shell=True)
  if check_if_cracked(command):
    is_cracked = True
  if is_cracked:
    print(" Password is cracked!!!")
    cracked_password = subprocess.check_output([command + " --show"], shell=True).decode()
    cracked_password = cracked_password.split(':')[-1]
    print(f" Password : {cracked_password}")
    print(f" Wordlist used : {wordlist}")
    break
  else:
    continue
  
if is_cracked == False:
  print(" Password was not found in charlist.")
else:
  pass